In [1]:
#Obtener el Top 200 semanal para un país en particular 
#Test 1: Japon 
#Instalar libreria de Spotify

In [2]:
pip install spotipy --upgrade

     |████████████████████████████████| 61kB 3.2MB/s 
     |████████████████████████████████| 153kB 18.3MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [3]:
#Credenciales Spotify
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="cc9a7bb857eb4282b846a1fe077003c4",
                                                           client_secret="817188bf767444ed8f6ca6f4080db1af"))

In [4]:
#Funciones para extraer informacion de Spotify 
def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        return items[0]
    else:
        return None


def get_artist_album_ids(name):
	artist = get_artist(name)
	results = sp.artist_albums(artist['id'], album_type='album', country='US')
	albums = results['items']
	album_ids = []
	for album in albums:
		album_ids.append(album['id'])

	return album_ids
def get_artist_features(name, feature='valence'):
	# features: valence (default), acousticness, danceability,
	#           energy, instrumentalness, speechiness, & tempo
	album_ids = get_artist_album_ids(name)
	features = []
	for album_id in album_ids:
		results = sp.album_tracks(album_id)
		tracks = results['items']

		for track in tracks:
			audioFeatures = sp.audio_features(track['id'])
			features.append(audioFeatures[0][feature])

	return features


In [38]:
#Funciones para obtener el Top200
import io
import pandas as pd
import requests
import time
import tqdm


def week_dates(date, weekday=0):
    week_start = date - pd.DateOffset(weekday=weekday, weeks=1)
    week_end = date + pd.DateOffset(weekday=weekday, weeks=0)
    return week_start, week_end


def get_chart(date, region='en', freq='weekly', chart='top200'):
    chart = 'regional' if chart == 'top200' else 'viral'
    date = pd.to_datetime(date)
    if date.year < 2017:
        raise ValueError('No chart data available from before 2017')
    if freq == 'weekly':
        start, end = week_dates(date, weekday=4)
        date = f'{start.date()}--{end.date()}'
    else:
        date = f'{date.date()}'
    url = f'https://spotifycharts.com/{chart}/{region}/{freq}/{date}/download'
    data = io.StringIO(requests.get(url).text)
    try:
        df = pd.read_csv(data, skiprows=1) # Fix Spotify's Note
    except pd.errors.ParserError:
        df = None
        print('No hay datos')
    return df
    #return url
    #return data


def get_charts(start, end, region='en', freq='daily', chart='top200', sleep=1):
    sample = 'D' if freq == 'daily' else 'W'
    dfs = []
    for date in tqdm.tqdm(pd.date_range(start=start, end=end, freq=sample)):
        df = get_chart(date, region=region, freq=freq, chart=chart)
        if df is not None:
            df['date'] = date
            dfs.append(df)
            time.sleep(sleep)
    return pd.concat(dfs)

In [54]:
#Positividad para un pais entre semanas
import pandas as pd
start_date = '2021-03-16'
end_date = '2021-03-25'
daterange = pd.date_range(start_date, end_date, freq='W-THU')
#Artistas = []
#Canciones = []
positividad = []
for single_date in daterange:
    date = single_date.strftime("%Y-%m-%d")
    df1 = get_chart(date, region='jp')
    Artistas = df1['Artist'].values
    Canciones = df1['Track Name'].values
    for i in range(len(Artistas)):
      query = Canciones[i] + '&' + Artistas[i]
      results = sp.search(q=query , type= 'track', limit=3, offset=0)
      #No todas las canciones tienen positividad identificada, por ello toca 
      #Poner una restriccion sobre la busqueda 
      b =  results['tracks']['items']
      if (len(b) == 0):
        continue
      else:
        a = results['tracks']['items'][0]['id']
        positividad.append(sp.audio_features(a)[0]['valence'])

    

In [55]:
len(positividad)

333

In [56]:
import numpy as np
positividad= np.array(positividad)
positividad.mean()

0.5451861861861862